In [51]:
from functools import reduce
from operator import mul

In [52]:
with open ('.\inputDay16.txt') as file:
    data_res = file.readlines()

In [53]:
def bits_from_hex(hex_value):
    bits = bin(int(hex_value, 16))[2:]
    len_data = len(bits)
    if len_data % 4 > 0:
        bits = "0" * (4 - len_data % 4) + bits
    return bits

In [54]:
data_res_bin = bits_from_hex(data_res[0])

In [55]:
def build_the_tree(binInput):
    packet = {
        'version': int(binInput[0:3],2),
        'type_ID': int(binInput[3:6],2),
        'children_info': []
    }
    if packet['type_ID'] == 4:
        packet['value'], remaining_bits =  get_literal_value(binInput[6:])
        return packet, remaining_bits
    return get_sub_packets(binInput, packet)


Get the literal values:

In [56]:
def get_literal_value(binInput):
    remaining_bits = binInput
    not_last_bits = True
    bin_digits = ""
    while not_last_bits:
        not_last_bits = remaining_bits[0] != '0'
        digit_chunk = remaining_bits[1:5]
        remaining_bits = remaining_bits[5:]
        bin_digits = bin_digits + digit_chunk
    dec_var = int(bin_digits, 2)
    return dec_var, remaining_bits

Get the subpackets. Those with 4 don't have one

In [57]:
def get_sub_packets(ori_bits, packet):
    len_type_id = ori_bits[6]
    if len_type_id == '0':
        len_subpackets = int(ori_bits[7:22],2)
        sub_packet_remaining = ori_bits[22:22+len_subpackets]
        bit_remaining = ori_bits[22+len_subpackets:]
        while len(sub_packet_remaining) > 0:
            sub_packet , sub_packet_remaining = build_the_tree(sub_packet_remaining)
            packet['children_info'].append(sub_packet)
    else:
        num_subpackets = int(ori_bits[7:18],2)
        bit_remaining = ori_bits[18:]
        for _ in range(num_subpackets):
            sub_packet, bit_remaining = build_the_tree(bit_remaining)
            packet['children_info'].append(sub_packet)
    return packet, bit_remaining

In [58]:
tree_info, _ = build_the_tree(data_res_bin)

In [59]:
def get_version_sum(packet_tree):
    return (
        packet_tree["version"]
        + sum(get_version_sum(child) for child in packet_tree["children_info"])
    )

In [60]:
get_version_sum(tree_info)

947

In [61]:
def get_value(packet_tree):
    if packet_tree["type_ID"] == 0:
        return sum(get_value(child) for child in packet_tree["children_info"])
    if packet_tree["type_ID"] == 1:
        return reduce(mul, (get_value(child) for child in packet_tree["children_info"]), 1)
    if packet_tree["type_ID"] == 2:
        return min(get_value(child) for child in packet_tree["children_info"])
    if packet_tree["type_ID"] == 3:
        return max(get_value(child) for child in packet_tree["children_info"])
    if packet_tree["type_ID"] == 4:
        return packet_tree["value"]
    if packet_tree["type_ID"] == 5:
        return int(get_value(packet_tree["children_info"][0])
                   > get_value(packet_tree["children_info"][1]))
    if packet_tree["type_ID"] == 6:
        return int(get_value(packet_tree["children_info"][0])
                   < get_value(packet_tree["children_info"][1]))
    if packet_tree["type_ID"] == 7:
        return int(get_value(packet_tree["children_info"][0])
                   == get_value(packet_tree["children_info"][1]))
    raise RuntimeError(f"Invalid type id {packet_tree['type_ID']}")

In [62]:
get_value(tree_info)

660797830937